---
# IESO (Power) Data Analysis and Preparation
---

## Setup

In [2]:
# Libraries:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# set paths to data files
IESO_path = 'https://raw.githubusercontent.com/VernonNaidoo-Toronto/3253_ML_Group_7_Electricity_Project/master/RAW_DATA/IESO_Data/'
weather_path = 'https://raw.githubusercontent.com/VernonNaidoo-Toronto/3253_ML_Group_7_Electricity_Project/master/DATA/Weather_Data/'

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# Set default plot styles
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = (16, 6)
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['axes.labelsize'] = 14

## Load Ontario Power DEMAND DATA

### Load files, concatenate, add date index to create hourly DataFrame: **power_supply_hourly**

In [4]:
#2017 Zonal Demand File: PUB_DemandZonal_2017.csv
demand_2017 = pd.read_csv(IESO_path+'PUB_DemandZonal_2017.csv', skiprows=3, parse_dates=['Date'])
demand_2018 = pd.read_csv(IESO_path+'PUB_DemandZonal_2018.csv', skiprows=3, parse_dates=['Date'])
demand_2019 = pd.read_csv(IESO_path+'PUB_DemandZonal_2019.csv', skiprows=3, parse_dates=['Date'])

# Print summary of records loaded:
for year, df in [[2017,demand_2017], [2018,demand_2018], [2019,demand_2019]]:
  print(f'Loaded {df.shape[0]} records and {df.shape[1]} columns from the {year} file.')

Loaded 8760 records and 15 columns from the 2017 file.
Loaded 8760 records and 15 columns from the 2018 file.
Loaded 8760 records and 15 columns from the 2019 file.


In [7]:
# Concatenate DataFrames
power_demand_hourly = pd.concat([demand_2017, demand_2018, demand_2019], ignore_index=True) 

# Change column names to Python standard lowercase:
cols = [column_name.lower() for column_name in power_demand_hourly.columns] 
power_demand_hourly.columns = cols

,date,hour,ontario demand,northwest,northeast,ottawa,east,toronto,essa,bruce,southwest,niagara,west,zone total,diff
0,2017-01-01,1,13522,495,1476,1051,1203,5665,1045,72,2986,465,1334,15790,"2,268"
1,2017-01-01,2,13117,495,1476,1051,1203,5665,1045,72,2986,465,1334,15790,"2,673"
2,2017-01-01,3,12816,495,1476,1051,1203,5665,1045,72,2986,465,1334,15790,"2,974"
3,2017-01-01,4,12605,495,1476,1051,1203,5665,1045,72,2986,465,1334,15790,"3,186"
4,2017-01-01,5,12563,495,1476,1051,1203,5665,1045,72,2986,465,1334,15790,"3,227"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26275,2019-12-31,20,15909,572,1274,1116,1030,5785,1130,64,3096,484,1466,16019,110
26276,2019-12-31,21,15198,563,1268,1064,986,5560,1078,64,2913,466,1418,15380,182
26277,2019-12-31,22,14678,557,1245,1036,945,5359,1036,65,2836,450,1392,14921,242
26278,2019-12-31,23,14136,541,1234,984,930,5138,994,64,2702,434,1354,14375,239


In [8]:
#Move date column to index
power_demand_hourly.set_index('date', inplace=True)

# Show sample records
print('Shape:',power_demand_hourly.shape)
display(power_demand_hourly.sample(5))

Shape: (26280, 14)


,hour,ontario demand,northwest,northeast,ottawa,east,toronto,essa,bruce,southwest,niagara,west,zone total,diff
date,,,,,,,,,,,,,,
2017-08-05,24,12240,405,1066,589,949,4343,649,56,2445,355,1344,12198,-42
2017-07-14,14,17067,380,1072,874,1084,6720,843,53,3575,575,1874,17049,-18
2018-08-15,14,21014,372,904,1300,921,8844,1298,118,4394,746,2219,21115,101
2018-05-06,3,10541,456,966,699,594,3775,561,65,2081,268,1073,10538,-3
2018-05-05,14,12133,482,941,860,410,5067,590,79,2464,355,1130,12377,244


### Data Preparation

#### Check for missing values; check datatypes; add day column

In [9]:
power_demand_hourly.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26280 entries, 2017-01-01 to 2019-12-31
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   hour            26280 non-null  int64 
 1   ontario demand  26280 non-null  int64 
 2   northwest       26280 non-null  int64 
 3   northeast       26280 non-null  int64 
 4   ottawa          26280 non-null  int64 
 5   east            26280 non-null  int64 
 6   toronto         26280 non-null  int64 
 7   essa            26280 non-null  int64 
 8   bruce           26280 non-null  int64 
 9   southwest       26280 non-null  int64 
 10  niagara         26280 non-null  int64 
 11  west            26280 non-null  int64 
 12  zone total      26280 non-null  int64 
 13  diff            26280 non-null  object
dtypes: int64(13), object(1)
memory usage: 3.0+ MB


In [10]:
# Check for missing values: NONE FOUND
col_count = 0 # count columns with nulls
for col in power_demand_hourly.columns:
  null_count = power_demand_hourly[col].isna().sum()
  if null_count >0:
    print(f'The {col} column has {null_count} rows without values.')
    col_count+=1
print(f'{col_count} columns have some missing values.')

0 columns have some missing values.


In [11]:
# Add weekday column
power_demand_hourly['day'] = power_demand_hourly.index.strftime('%A')
power_demand_hourly.head(3)

,hour,ontario demand,northwest,northeast,ottawa,east,toronto,essa,bruce,southwest,niagara,west,zone total,diff,day
date,,,,,,,,,,,,,,,
2017-01-01,1,13522,495,1476,1051,1203,5665,1045,72,2986,465,1334,15790,"2,268",Sunday
2017-01-01,2,13117,495,1476,1051,1203,5665,1045,72,2986,465,1334,15790,"2,673",Sunday
2017-01-01,3,12816,495,1476,1051,1203,5665,1045,72,2986,465,1334,15790,"2,974",Sunday


### Export CSV Files for Machine Learning

Output Files:
- **power_demand_hourly**
- **power_demand_daily**

#### Create and export power_demand_hourly.csv

In [12]:
#Save as csv:
filename_with_path = 'power_demand_hourly.csv'
power_demand_hourly.to_csv(filename_with_path)

In [13]:
#Test:
filename_with_path = 'power_demand_hourly.csv'
test = pd.read_csv(filename_with_path)
test.set_index(['date']).head(3)

,hour,ontario demand,northwest,northeast,ottawa,east,toronto,essa,bruce,southwest,niagara,west,zone total,diff,day
date,,,,,,,,,,,,,,,
2017-01-01,1,13522,495,1476,1051,1203,5665,1045,72,2986,465,1334,15790,"2,268",Sunday
2017-01-01,2,13117,495,1476,1051,1203,5665,1045,72,2986,465,1334,15790,"2,673",Sunday
2017-01-01,3,12816,495,1476,1051,1203,5665,1045,72,2986,465,1334,15790,"2,974",Sunday


#### Create and export power_demand_daily.csv

In [22]:
# Resample to daily totals
power_demand_daily = power_demand_hourly.resample('D').sum()

# Drop unwanted columns:
power_demand_daily.drop(columns='hour', inplace=True)

# Add weekday column
power_demand_daily['day'] = power_demand_daily.index.strftime('%A')
power_demand_daily.head(3)

,ontario demand,northwest,northeast,ottawa,east,toronto,essa,bruce,southwest,niagara,west,zone total,day
date,,,,,,,,,,,,,
2017-01-01,334736,11880,35424,25224,28872,135960,25080,1728,71664,11160,32016,378960,Sunday
2017-01-02,354759,11569,34684,22220,29166,139493,24299,1694,74650,11394,34086,383256,Monday
2017-01-03,388840,11386,34200,20581,29135,140783,23733,1668,76095,11489,35049,384118,Tuesday


In [23]:
#Save as csv:
filename_with_path = 'power_demand_daily.csv'
power_demand_daily.to_csv(filename_with_path)

In [25]:
#Test:
filename_with_path = 'power_demand_daily.csv'
test = pd.read_csv(filename_with_path)
test.set_index(['date']).head(3)

,ontario demand,northwest,northeast,ottawa,east,toronto,essa,bruce,southwest,niagara,west,zone total,day
date,,,,,,,,,,,,,
2017-01-01,334736,11880,35424,25224,28872,135960,25080,1728,71664,11160,32016,378960,Sunday
2017-01-02,354759,11569,34684,22220,29166,139493,24299,1694,74650,11394,34086,383256,Monday
2017-01-03,388840,11386,34200,20581,29135,140783,23733,1668,76095,11489,35049,384118,Tuesday
